# Multi-Agent

              +----------------+
              | Manager agent  |
              +----------------+
                       |
        _______________|______________
       |                              |
Code Interpreter            +------------------+
    tool                    | Web Search agent |
                            +------------------+
                               |            |
                        Web Search tool     |
                                   Visit webpage tool

In [1]:
! pip install markdownify duckduckgo-search smolagents --upgrade -q

In [5]:
from huggingface_hub import login

In [7]:
model_id = "Qwen/Qwen2.5-Coder-32B-Instruct"

## Web search tool

In [16]:
import re
import requests
from markdownify import markdownify
from requests.exceptions import RequestException
from smolagents import tool
import pandas as pd

@tool
def visit_webpage(url: str) -> str:
    """Visits a webpage at the given URL and returns its content as a markdown string.

    Args:
        url: The URL of the webpage to visit.

    Returns:
        The content of the webpage converted to Markdown, or an error message if the request fails.
    """
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes

        # Convert the HTML content to Markdown
        markdown_content = markdownify(response.text).strip()

        # Remove multiple line breaks
        markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)
        # Remove markdown headers
        markdown_content = re.sub(r"^#+ ", "", markdown_content) 

        return markdown_content

    except RequestException as e:
        return f"Error fetching the webpage: {str(e)}"
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"

In [9]:
print(visit_webpage("https://en.wikipedia.org/wiki/Hugging_Face")[:500])

Hugging Face - Wikipedia

[Jump to content](#bodyContent)

Main menu

Main menu

move to sidebar
hide

Navigation

* [Main page](/wiki/Main_Page "Visit the main page [z]")
* [Contents](/wiki/Wikipedia:Contents "Guides to browsing Wikipedia")
* [Current events](/wiki/Portal:Current_events "Articles related to current events")
* [Random article](/wiki/Special:Random "Visit a randomly selected article [x]")
* [About Wikipedia](/wiki/Wikipedia:About "Learn about Wikipedia and how it works")
* [Conta


# Multi-Agent System


In [17]:
from smolagents import (
    CodeAgent,
    ToolCallingAgent,
    HfApiModel,
    DuckDuckGoSearchTool,
    LiteLLMModel,
)

model = HfApiModel(model_id)

web_agent = ToolCallingAgent(
    tools=[DuckDuckGoSearchTool(), visit_webpage],
    model=model,
    max_steps=1,
    name="web_search_agent",
    description="Runs web searches for you.",
)

In [18]:
manager_agent = CodeAgent(
    tools=[],
    model=model,
    managed_agents=[web_agent],
    additional_authorized_imports=["time", "numpy", "pandas", "json"],
)

In [19]:
wine_data = manager_agent.run("Make a comparative table of all wine cépages for red wine in France with the following information: name, color, grape variety, region, price, and rating.")


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Make a comparative table of all wine cépages for red wine in France with the following information: name,       │
│ color, grape variety, region, price, and rating.                                                                │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  wine_cepages_search = web_search_agent(task="Provide a detailed list of red wine cépages in France, including    
  details about grape variety, region, price, and rating.")                                                        
  print(wine_cepages_search)                                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭────────────────────────────────────────── New run - web_search_agent ───────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'web_search_agent'.                                                                │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ Provide a detailed list of red wine cépages in France, including details about grape variety, region, price,    │
│ and rating.                                                                                                     │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

[Step 3: Duration 8.87 seconds| Input tokens: 10,882 | Output tokens: 2,047]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(wine_cepages_table)                                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer:               name color      grape_variety                                                
region                                                     price                                   rating
Cabernet Sauvignon   Red Cabernet Sauvignon                        Bordeaux, Médoc, Saint-Émilion                  
€10 to over €1,000                                      90+
            Merlot   Red             Merlot                      Bordeaux, Saint-Émilion, Pomerol                  
€10 to €200                                    80-90
      Syrah/Shiraz   Red       Syrah/Shiraz         Côtes du Rhône, Châteauneuf-du-Pape, Gigondas                  
€15 to €500                                    85-95
         Mourvèdre   Red          Mourvèdre Côtes du Rhône, Gigondas, Châteauneuf-du-Pape, Bandol                  
€20 to €300                                    85-95
             Gamay   Red              Gamay                                            Beaujolais                  
€5 to €40                                    75-85
        Pinot Noir   Red         Pinot Noir                  Burgundy, Gevrey-Chambertin, Pommard €35 to €500 for 
village-level wines, much higher for Crus                                      90+
            Malbec   Red             Malbec                          Languedoc-Roussillon, Cahors                  
€25 to €150 80-90, with some top-tier Malbecs higher

[Step 4: Duration 8.35 seconds| Input tokens: 17,067 | Output tokens: 2,094]

In [30]:
## Read a json cepages.json
with open('cepages.json', 'r') as f:
    cepages = json.load(f)

wine_table = pd.DataFrame(cepages)

# Display the table
wine_table.head()

# Save to CSV
wine_table.to_csv("wine_comparative_table.csv", index=False)

print("Table saved as 'wine_comparative_table.csv'")

# Use the web_agent to get the data
wine_data = web_agent.run("Make a comparative table of all wine cépages for red wine in France with the following information: name.")

# Convert the wine_data text to a DataFrame
wine_table = pd.read_csv(StringIO(str(wine_data)), sep="|")



Table saved as 'wine_comparative_table.csv'


╭────────────────────────────────────────── New run - web_search_agent ───────────────────────────────────────────╮
│                                                                                                                 │
│ Make a comparative table of all wine cépages for red wine in France with the following information: name.       │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'web_search' with arguments: {'query': 'List of red wine cépages in France'}                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: ## Search Results

|French Red Wines: 101 Red Wine Types in France | TasteAtlas](https://www.tasteatlas.com/france/red-wine)
Garnacha, also known as Grenache in France, is a widely cultivated red wine grape variety known for producing wines
with rich, fruity flavors and high alcohol content. It is one of the most versatile and widely p... READ MORE. 40. 
Wine Appellation. Saint-Julien. Saint-Julien-Beychevelle. n/a.

|100 Best Red Wines in France - TasteAtlas](https://www.tasteatlas.com/best-rated-red-wines-in-france)
Chambertin is a French appellation and a Grand Cru vineyard located in Burgundy, more precisely in the Côte de 
Nuits (Côte d'Or) wine region. The designation produces red wines that are mostly varietal and based on Pinot Noir.
They typically display complex aromas of red and dark fruit that are often complemented by smoky notes of spices 
and licorice, as well as subtle herbal and floral nuances.

|Liste des cépages - Vin-Vigne.com](http://www.vin-vigne.com/cepage/)
En 2011, la France comptait plus de 785 000 hectares de vignes sur plus de 87 000 exploitations viticoles 
(domaines, châteaux, coopératives, caves particulières…). Ces vignes sont composées essentiellement (près de 70%) 
des cépages rouges : merlot, grenache noir, cabernet sauvignon, pinot noir, carignan, poulsard et trousseau ; des 
cépages blancs : ugni blanc, chardonnay, sauvigon et ...

|From Pinot Noir to Cabernet Sauvignon: A French Red Wine Guide - WineO 
Mark](https://www.wineomark.com/french-red-wine-guide/)
The French wine classification system, also known as the appellation d'origine contrôlée (AOC), is a legal 
framework that defines the geographic origin, grape varieties, and winemaking practices for wines produced in 
France.In addition, the AOC system was established in 1935 to protect the quality and authenticity of French wines 
and has since become a model for wine regions around the world.

|Les cépages français : Guide complet des variétés de raisin utilisées 
...](https://www.vinadomus.fr/content/les-cepages-francais-guide-complet)
Explorez les cépages utilisés dans les vins français, du Pinot Noir au Chardonnay, en passant par les cépages moins
connus. Découvrez leur rôle, leurs arômes, et les meilleures régions de production. ... La France, avec ses 806 000
hectares de vignes, est le pays de référence en matière de diversité viticole.

|French Red Wine (Types, Prices, Best Wines to Buy in 2025) - 
Vinovest](https://www.vinovest.co/blog/french-red-wine)
A. 2016 Domaine du Pelican Arbois Trois Cepages, Jura. This French red contains a red blend of Pinot Noir, Poulsard
and Trousseau. It is a light and perfumed wine with a mild flavor of berries, and aromas of wild herbs. ... South 
West France is famous for its red wines. Some of the red wine grape varieties grown in the region include Cabernet 
...

|Les cépages de France](https://www.vindefrance.com/cepages-de-france)
Ces variétés de raisins qui font la renommée des Vins De France. Comme pour de nombreux fruits, les variétés de 
vigne produisent des raisins aux styles et aux goûts différents. Pour le plaisir de chaque consommateur de vin. Les
plus connus dans le monde sont originaires de France. Merlot, Chardonnay, Sauvignon Blanc ou Syrah : voilà quelques
noms de cépages bien français connus dans ...

|Liste des Cépages de Vin de A à Z - WineSelection.fr](https://www.wineselection.fr/cepages/)
Découvrez le listing alphabétique des appellations de vin en France et à l'étranger. Tous les cépages de A à Z pour
découvrir les vins autrement. ... NOTRE WINE COLLECTIF. En savoir plus. NOS 7 ENGAGEMENTS. En savoir plus. LES 
CHEFS SOMMELIERS.

|The different types of grape varieties in France - 
Oeni](https://oeni.app/en/les-differents-types-de-cepages-en-france/)
In France, it is mainly grown in the Bordeaux region, where it makes up an essential part of the region's famous 
blends. Caractéristiques : Cabernet Sauvignon produces powerful, tannic wines with aromas of bla

[Step 1: Duration 2.98 seconds| Input tokens: 1,298 | Output tokens: 28]

Reached max steps.

[Step 2: Duration 25.70 seconds| Input tokens: 2,678 | Output tokens: 493]

AttributeError: module 'pandas' has no attribute 'StringIO'

In [25]:
# Save to CSV
wine_table.to_csv("wine_comparative_table.csv", index=False)

print("Table saved as 'wine_comparative_table.csv'")

NameError: name 'wine_table' is not defined